In [1]:
import pandas as pd
import uuid
import random
import json

### Fix on Google Drive

In [12]:
df = pd.read_pickle('../data/teaching.pkl')
sermons = df[df.category=='sermon'].rename(columns={'title':'title_raw'})
def get_title(s):
    try:
        return s[:s.rfind('(')].strip()
    except:
        return ''

def get_passage(s):
    try:
        return s[s.rfind('('):].strip().replace('(','').replace(')','')
    except:
        return ''
                
sermons['title'] = sermons.title_raw.apply(lambda x: get_title(x))
sermons['passage'] = sermons.title_raw.apply(lambda x: get_passage(x))
sermons = sermons.sort_values('date_str').reset_index(drop=True)
sermons.to_excel('../data/temp_sermons_for_google_drive.xlsx')

### Prep for Sanity schema

In [2]:
def new_audio_link(audioLink):
    try:
        return audioLink.split('.org/')[-1].replace('/','___')
    except:
        return ""

In [11]:
sermons_edited = pd.read_excel('../data/sermons.xlsx')
sermons_edited = sermons_edited.drop(columns=['Unnamed: 0',])
sermons_edited['mp3File'] = sermons_edited.audioLink.apply(lambda x: new_audio_link(x))
sermons_edited = sermons_edited.drop(columns=['date'])
sermons_edited = sermons_edited.rename(columns={'date_str':'date'})
sermons_edited = sermons_edited.reset_index().rename(columns={'index':'_id', 'series':'sermonSeries'})
sermons_edited['_id'] = [str(uuid.uuid1(random.randint(0,100000))) for i in range(sermons_edited.shape[0])]
sermons_edited['_type'] = 'sermons'
sermons_edited = sermons_edited[['_id', '_type', 'date', 'speaker', 'book', 'passage', 'title', 'sermonSeries', 'mp3File']]
sermons_edited = sermons_edited.fillna("")

In [12]:
sermons_edited = sermons_edited[sermons_edited.date <= "2022-10-31"]

sermons_edited['book'] = sermons_edited.book.apply(lambda x: 'Hebrews' if x == 'Hebrew' else x)

sermons_edited.drop(columns=['mp3File']).to_json("../cms/sermons.ndjson",
           orient="records",
           force_ascii=False,
           lines=True)


In [16]:
mp3List = sermons_edited[['_id','mp3File']].rename(columns={'_id':'assetId', 'mp3File':'mp3FileName'}).to_dict('records')
with open('../cms/mp3List.js', 'w') as f:
    f.write(f"export const mp3s = {json.dumps(mp3List)}")